In [5]:
import pandas as pd
import logging
from typing import List
import numpy as np

In [6]:
import gc


In [3]:
stata_data = pd.read_stata('/home/stanoo/dcrt/data/DTAS/all_xlsx_files.dta')

In [4]:
to_keep = ['court', 'line', 'date_dd', 'date_mon', 'date_yyyy', 'caseid_type',
       'caseid_no', 'filed_dd', 'filed_mon', 'filed_yyyy', 'origcourt',
       'appeal_type', 'appeal_no', 'appeal_yyyy', 'case_type', 'bench_1',
       'bench_2', 'bench_3', 'bench_4', 'bench_5', 'bench_6', 'bench_7',
       'comingfor', 'outcome', 'reason_adj', 'next_dd', 'next_mon',
       'next_yyyy', 'firstparty_m', 'firstparty_f', 'firstparty_org',
       'secondparty_m', 'secondparty_f', 'secondparty_o', 'legalrep',
       'p_witness', 'd_witness', 'custody', 'other_details']
df = stata_data[to_keep]

In [7]:
del stata_data
gc.collect()

24862

In [10]:
# drop df['line'] column
df.drop('line', axis=1, inplace=True)

In [11]:
df.columns

Index(['court', 'date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'origcourt', 'appeal_type',
       'appeal_no', 'appeal_yyyy', 'case_type', 'bench_1', 'bench_2',
       'bench_3', 'bench_4', 'bench_5', 'bench_6', 'bench_7', 'comingfor',
       'outcome', 'reason_adj', 'next_dd', 'next_mon', 'next_yyyy',
       'firstparty_m', 'firstparty_f', 'firstparty_org', 'secondparty_m',
       'secondparty_f', 'secondparty_o', 'legalrep', 'p_witness', 'd_witness',
       'custody', 'other_details'],
      dtype='object')

### Indicators 
#### 1 Appeals Judgements
#### 2 Certificates of urgency
#### 3 Injunctions
#### 4 Remand days
#### 5 First hearing
#### 6 Taxation - Advocate-client Bc 


In [17]:
file_path = '/home/stanoo/dcrt/data'
df = pd.read_csv(f'{file_path}/processed_23-24_df.csv')

/tmp/ipykernel_32489/2985956275.py:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}/processed_23-24_df.csv')


In [18]:
hc_stations = ['Marsabit', 'Embu', 'Chuka', 'Muranga']
sub_df = df[df['court'].isin(hc_stations)]

In [19]:
sub_df.shape

(20681, 50)

## not certified application/injuctions

In [31]:
condition = (
    sub_df['comingfor'].isin(['Notice of Motion', 'Hearing of Applications', 'Hearing- Application', 'Ruling on Applications']) |
    sub_df['outcome'].isin(['Not Certified Urgent', 'Not  Certified Urgent', 'Application Heard'])
)
sub_df = sub_df.copy()

sub_df['other_applications'] = np.where(condition, 1, 0)
sub_df['application'] = sub_df.groupby('case_number')['other_applications'].transform('max')
applic_sub_df = sub_df[sub_df['application'] == 1]
applic_sub_df.to_csv(f'{file_path}/hc_application_data.csv', index=False)

applic_sub_df.groupby(['court', 'case_number'])['other_applications'].sum().sort_values(ascending=False).reset_index(name='count').sort_values(by=['court', 'count'], ascending=[True, False]).reset_index(drop=True).to_csv(f'{file_path}/hc_application_cases.csv', index=False)

## Certified Urgent applications

In [ ]:
cert_df = sub_df[sub_df['application'] == 0]
cert_df.loc[:, 'urgent'] = cert_df['outcome'].apply(lambda x: 1 if x == 'Certified Urgent' else 0)
cert_df.loc[:, 'certified'] = cert_df.groupby('case_number')['urgent'].transform('max')
urgent_cases = cert_df[cert_df['certified'] == 1]
urgent_cases.groupby(['court','case_number'])['certified'].count().reset_index(name='count').to_csv(f'{file_path}/hc_urgent_cases.csv', index=False)
urgent_cases.to_csv(f'{file_path}/hc_certificate_data.csv', index=False)

## Remand Days

In [20]:
custody_df = sub_df[(sub_df['custody'] == 1) & (sub_df['broad_case_type'] == 'Murder')]

In [15]:
custody_df = sub_df[(sub_df['custody'] == 1) & (sub_df['broad_case_type'] == 'Murder')]
custody_df.loc[:, 'reg'] = custody_df.groupby('case_number')['registered'].transform('max')
custody_df = custody_df.copy()
custody_df = custody_df[custody_df['reg'] == 1]

/tmp/ipykernel_32489/1720438997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custody_df.loc[:, 'reg'] = custody_df.groupby('case_number')['registered'].transform('max')


In [24]:
custody_df.loc[:, 'reg'] = custody_df.groupby('case_number')['registered'].transform('max')

In [27]:
custody_df = custody_df[custody_df['resolved'] != 1]

In [29]:
custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False).reset_index(name='count').to_csv(f'{file_path}/hc_custody_cases.csv', index=False)
custody_df.to_csv(f'{file_path}/hc_custody_data.csv', index=False)

In [10]:

custody_df['resolved'] = custody_df.groupby('case_number')['concluded'].transform('max')
custody_df = custody_df[custody_df['resolved'] != 1]
custody_df.groupby(['court', 'case_type'])['reg'].count().reset_index(name='count')
custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False).reset_index(name='count').to_csv(f'{file_path}/hc_custody_cases.csv', index=False)
custody_df.to_csv(f'{file_path}/hc_custody_data.csv', index=False)

/tmp/ipykernel_32489/198743014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custody_df.loc[:, 'reg'] = custody_df.groupby('case_number')['registered'].transform('max')


In [13]:
custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False)

Series([], Name: reg, dtype: int64)

## Appeals

In [50]:
# not certified application/injuctions
jd_condition = (
    sub_df['comingfor'].isin(['Judgment', 'Ruling']) &
    sub_df['outcome'].isin(["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"])
)
sub_df = sub_df.copy()

sub_df['delivered'] = np.where(jd_condition, 1, 0)

appeals = sub_df[sub_df['broad_case_type'].isin(['Civil Appeal', 'Criminal Appeal'])]
appeals = appeals.copy()
appeals = appeals[appeals['delivered'] == 1]
appeals['judgement'] = appeals.groupby('case_number')['delivered'].transform('max')
appeals.to_csv(f'{file_path}/hc_appeal_data.csv', index=False)
(appeals.groupby(['court', 'case_number'])['judgement'].sum()
.sort_values(ascending=False).reset_index(name='count')
.sort_values(by=['court', 'count'], ascending=[True, False]).reset_index(drop=True)
.to_csv(f'{file_path}/hc_appeal_cases.csv', index=False))

In [ ]:
(appeals.groupby(['court', 'case_number'])['judgement'].sum()
.sort_values(ascending=False).reset_index(name='count')
.sort_values(by=['court', 'count'], ascending=[True, False])
.reset_index(drop=True))


## Taxations

In [53]:
taxation_condition  = sub_df['comingfor'].isin(['Taxation and Issuance of Certificates',
'Signing of Taxation''Notice of Taxation',
'Filling Bill of Cost',
'Mention of Bill of Cost',
'Taxation''Assessment of costs/Taxation',
'Assessment of costs']) | sub_df['outcome'].isin(['Taxation Date Set'])

sub_df = sub_df.copy()

sub_df['taxation'] = np.where(taxation_condition, 1, 0)
sub_df.loc[:, 'boc'] = sub_df.groupby('case_number')['taxation'].transform('max')
sub_df['resolved'] = sub_df.groupby('case_number')['concluded'].transform('max')
taxation_df = sub_df[(sub_df['boc'] == 1) & (sub_df['broad_case_type'] =='Civil Application')]
resolved_boc = taxation_df[taxation_df['resolved'] == 1]
resolved_boc.to_csv(f'{file_path}/hc_taxation_data.csv', index=False)
resolved_boc.groupby(['court', 'case_number'])['taxation'].sum().reset_index(name='count').to_csv(f'{file_path}/hc_taxation_cases.csv', index=False)

In [67]:
resolved_boc.groupby(['court', 'case_type'])['taxation'].sum().reset_index(name='count')

,court,case_type,count
0,Chuka,Civil Case Miscellaneous,2
1,Embu,Civil Case Miscellaneous,12
2,Marsabit,Civil Case Miscellaneous,1
3,Muranga,Civil Case Miscellaneous,10


In [ ]:
'Record Of Appeal Filed'

## First scheduled

In [ ]:
hearing_set =  ['Hearing Date Set (In Court)', 
'Hearing Date Set (At The Registry)', 
'Hearing Date Set',
'Hearing Date Given']
hearing_date = ['Hearing',
'Hearing - Appeals',
'Hearing- Main Suit',
'Fresh Hearing']

In [ ]:
hearing_condition = sub_df['comingfor'].isin(hearing_set) | sub_df['outcome'].isin(hearing_date)
sub_df = sub_df.copy()
sub_df['hearing'] = np.where(hearing_condition, 1, 0)

In [83]:
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [91]:
sub_df[sub_df['outcome'] == 'Directions Given'].groupby('comingfor')['comingfor'].count().sort_values(ascending=False).reset_index(name='count')

,comingfor,count
0,Mention,669
1,Certificate of urgency,229
2,Directions,113
3,Hearing of Applications,92
4,Hearing,66
5,Admission of Appeal,12
6,Dismissal,11
7,Notice to Show Cause (NTSC),11
8,Mention Date for compliance,10
9,Judgment,9


In [84]:
sub_df.groupby('outcome')['outcome'].count().sort_values(ascending=False).reset_index(name='count')

,outcome,count
0,Mention Date Set (In Court),8090
1,Case Registered/Filed,2912
2,Hearing Date Set (In Court),1757
3,Directions Given,1294
4,Ruling Delivered- Case Closed,938
5,Adjournment,825
6,Ruling Date Given,719
7,Judgment Date Given,563
8,Order Issued - Case Closed,479
9,Judgment Delivered- Case Closed,376


In [ ]:
sub_df.loc[:, 'hearing'] = sub_df.groupby('case_number')['hearing'].transform('max')
sub_df['resolved'] = sub_df.groupby('case_number')['concluded'].transform('max')

In [ ]:
hearing_df = sub_df[(sub_df['hearing'] == 1) & (sub_df['broad_case_type'] =='Civil Application')]
resolved_hearing = hearing_df[hearing_df['resolved'] == 1]
resolved_hearing.to_csv(f'{file_path}/hc_hearing_data.csv', index=False)
resolved_hearing.groupby(['court', 'case_number'])['hearing'].sum().reset_index(name='count').to_csv(f'{file_path}/hc_hearing_cases.csv', index=False)